# Dealing with deeper networks:



### Weight Initialisation


Suppose we have the weighted sum $y = \sum_{k=1}^{n} w_k x_k$. If $w_k, x_k$ are *independent*, then 
$$
    \text{Var}(y) = n\text{Var}(w)\text{Var}(x), \tag{1}
$$
where $n$ is the size of the vectors $w_k$ and $x_k$.

Consider one layer $(i)$ of a neural network, with:
- Weights $w_{jk}^{(i)}$ &mdash; denoting the weight of the connection from node $k$ of the previous layer, $(i-1)$, to node $j$ in layer $(i)$
- Activations $x_{k}^{(i)}$ &mdash; denoting the activation value of node $k$ at layer $i$, where $1 \leq k \leq n_{i}$ with $n_i$ being the number of nodes in layer $(i)$
- Activations $x_{j}^{(i+1)}$ &mdash; denoting the activation value of node $j$ at the next layer, where $1 \leq j \leq n_{i+1}$ with $n_{i+1}$ being the number of nodes in layer $(i+1)$

The activation value at each node in the next layer is given by:

$$
    x_j^{(i+1)} = \texttt{activation}(\sum_{k=1}^{n_i} w_{jk}^{(i)} x_{k}^{(i)}). \tag{2}
$$

From $(1)$ and $(2)$, we have 

$$
    \text{Var}(\sum_{k=1}^{n_i} w_{jk}^{(i)} x_{k}^{(i)}) = n_i \text{Var}(w_{jk}^{(i)}) \text{Var}(x_{k}^{(i)}),
$$
then substituting $\sum_{k=1}^{n_i} w_{jk}^{(i)} x_{k}^{(i)}$ for $x^{(i+1)}$,
$$
    \text{Var}(x^{(i+1)}) \approx G_0 n_i \text{Var}(w^{(i)}) \text{Var}(x^{(i)}), \tag{3}
$$
where $G_0$ is a constant to account for the activation function &mdash; normally we just set it to $G_0 = \frac{1}{2}$.

Suppose a network has $D$ layers, with input vector $x^{(i)}$ and output $z$. From $(3)$ collected across $D$ layers, we have

$$
    \text{Var}(z) \approx \underbrace{\prod_{i=1}^{D} \big( G_0 n_i \text{Var}(w_i) \big)}_{\text{We want this to be } \approx \space 1} \cdot \text{Var}(x). \tag{4}
$$

If the product in $(4)$ is less than 1, then the activations going from input towards output will exponentially decay. If the product in $(4)$ is greater than 1, then the activations will exponentially grow going from input layer to output layer.

Similarly, for backpropagation we have

$$
    \text{Var}(\frac{\partial}{\partial x}) \approx \underbrace{\prod_{i=1}^{D} \big( G_1 n_i \text{Var}(w_i) \big)}_{\text{We want this to be } \approx \space 1} \cdot \text{Var}(\frac{\partial}{\partial z}), \tag{5}
$$

where we want the differentials to also not exponentially grow/decay too much across layers.

The basis for deciding weights in *weight initialisation* is choosing weights $w_{jk}^{(i)}$ for all layers $i$ such that: 

$$
    G_1 n_i \text{Var}(w^{(i)}) = 1, \tag{6}
$$
where $G_1 = \frac{1}{2}$, usually.


#### Example of Weight Initialisation:
<img src="images/weight-initialisation-1.png" width="50%">
<em><p style="text-align: center;">Error on the y axis</p></em>

The above graph shows the difference in training speed for a 22-layer $\texttt{ReLU}$ network for when suboptimal initial weights are chosen (blue line) vs. when weights are chosen according to $(6)$.


### Batch Normalisation:

Batch normalistion is a technique for making deeper networks faster to train and stabler by recentering and rescaling the inputs to chosen layers in the network (normalisation) for each *mini-batch* .

- *Standardisation* &mdash; recentering and rescaling the mini-batch to have mean $\mu = 0$ and standard deviation $\sigma = 1$ by applying the linear transformation, $z=\frac{x-\mu_d}{\sigma_d}$, where $\mu_d$ and $\sigma_d$ are calculated from the current mini-batch.
    - Without standardising input values, having disproportionately large input values will cause the exploding gradient problem. A large activation value in an earlier layer will be transferred onto the next layer, cascading through the network and causing unstable weight changes
- *Covariate shift* &mdash; covariate shift is the change in the distribution of input values. This may happen, for instance, if the training set's items come from images downloaded from the web while the test set's training items comes from an iPhone camera, for instance.
    - The aim of batch normalisation is to reduce covariate shift by normalising the activations of each layer.
    - Batch normalisation reduces the impact that earlier layers have on the distribution of inputs into later hidden layers. Ie. this 'decouples' the later layer's parameters from the earlier's layer's parameters



 
Normalising the activations of each layer to have mean $\mu = 0$ and standard deviation $\sigma = 1$ may limit the expressive power of the network. Because of this, we introduce addtional trainable parameters $\gamma$ and $\beta$.



#### Algorithm:

Suppose we have a batch of size $m$



<table>
    <tr>
        <td width="60%">            
            <img src="images/batch-normalisation.png">
        </td>
        <td>
            <img src="images/batch-norm-gamma-beta.png">
        </td>
    <tr>
</table>


Where $\mu_{B}$ is the mean of the output predictions, $\sigma_{B}^2$ is the variance of the output predictions.

Each output prediction $\hat{x}_i$ is then normalised using the formula $\hat{x}_i = \frac{x_i-\mu_B}{\sqrt{\sigma_B^2 + \epsilon}}$ where $\epsilon$ is just a parameter to prevent zero division error. After this, all outgoing activation values have been standardised on this layer &mdash; $N(0, 1)$.

The final output takes this activation and 'calibrates' it with new learnable parameters $\gamma$ and $\beta$: $y_i = \gamma \hat{x}_i + \beta$.


- The $\gamma$ and $\beta$ for each neuron is often initialised to be $1$ and $0$ respectively

- This introduces more independent parameters to the network, but it keeps the activations within a 'healthy' range




### Resources:
- <a href="https://mlexplained.com/2018/01/10/an-intuitive-explanation-of-why-batch-normalization-really-works-normalization-in-deep-learning-part-1/">Intuitive explanation of why batch normalisation works</a>